# Data Cleanup Final File for Group 4

In [ ]:
#Dependencies
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import requests
import scipy
from scipy import stats

Rafael

In [ ]:
#Getting country codes
countrycodes_file = "Resources/CountryCodesLookup.csv"
countrycodes = pd.read_csv(countrycodes_file, encoding="ISO-8859-1")
countrycodes = countrycodes.rename(columns={'COUNTRY':'country'})

#Getting happiness index
happiness_file = "Resources/WorldHappinessReport.csv"
happiness = pd.read_csv(happiness_file, encoding="ISO-8859-1")

happiness2017 = happiness.loc[happiness['year']==2017,['country','Life Ladder']]
print(happiness2017.head())

#Merging both
mymerge = pd.merge(countrycodes, happiness2017, how="left", on=['country'])
mymerge = mymerge.drop(['A3 (UN)','NUM (UN)','DIALING CODE','spotifyFlag'],axis=1)
print(mymerge.head())

In [ ]:
#Checking indicators and grabbing interesting data for analysis
baseurl = "http://api.worldbank.org/v2/country/"
year = "2017"

def grabindicator(y):
    if y == 0: #Mobile cellular subscriptions (per 100 people)
        x = "IT.CEL.SETS.P2"
    elif y == 1: #Individuals using the Internet (% of population)
        x = "IT.NET.USER.ZS"
    elif y == 2: # Population, female (% of total)
        x = "SP.POP.TOTL.FE.ZS"
    elif y == 3: # Unemployment, total (% of total labor force) (modeled ILO estimate)
        x = "SL.UEM.TOTL.ZS"
    else:
        print(f'Indicator "{y}" not available')
    return x

temp = []
mobilephones = []
internet = []
femalepopulation = []
unemployment = []

countryfound = []

def lists(indicatorcod, x, lock, errorfree):

    try:   
        if indicatorcod == 0: #Mobile cellular subscriptions (per 100 people)
            temp = x[1]
            temp = temp[0]
            mobilephones.append(temp['value'])
    
        elif indicatorcod == 1: #Individuals using the Internet (% of population)
            temp = x[1]
            temp = temp[0]
            internet.append(temp['value'])         

        elif indicatorcod == 2: # # Population, female (% of total)
            temp = x[1]
            temp = temp[0]
            femalepopulation.append(temp['value'])

        elif indicatorcod == 3: # Unemployment, total (% of total labor force) (modeled ILO estimate)
            temp = x[1]
            temp = temp[0]
            print (temp)
            print(temp['value'])
            unemployment.append(temp['value'])       
        
        else:
            print("*** Error on function 'lists' ***")

        
        if lock != 1 and errorfree == 1 and indicatorcod <1:
                countryfound.append(pais)
                lock = 1
        
    except TypeError:
        print(f'Error: Country Code "{countrycode}" not found')



for i in range (4):
    indicatorcode = grabindicator(i)
    indicatorcounter = int(i)
    lock = 0

    for code,pais in zip(mymerge['A2 (ISO)'], mymerge['country']):        
        countrycode = code
        print(f'Iteration {indicatorcounter}')
        print(pais)
        query_url = f"{baseurl}{countrycode}/indicator/{indicatorcode}?date={year}&format=json"
        print (query_url)
        response = requests.get(query_url).json()
        print(f'response is {response}\n')
        
        if len(response) < 2:
            print(f'response for {countrycode} is invalid\n')
            errorfree = 0
        elif response[1] is None:
            print(f'response for {countrycode} is None \n')
            errorfree = 0
        else:
            errorfree = 1
            lists(indicatorcounter, response, lock, errorfree)
                                  

print("\n***********all data was collected via the API")

countryfound1 = pd.DataFrame(countryfound)

#Compiling grabbed data from API Calls into one data frame

mymerge_dict = {
    "Country": countryfound,
    "Individuals using the Internet (% of population)": internet,
    "% Female Total Population": femalepopulation,
    "Total Unemployment": unemployment,
    "Mobile Phones (per 100 people)": mobilephones}

mymerge_data = pd.DataFrame(mymerge_dict)

womenmajority = []
label1 =[]


for y in mymerge_data['% Female Total Population']:
   
    if y == 50:
        womenmajority.append('turquoise')
        label1.append("Country without gender majority")
    elif y < 50:
        womenmajority.append('yellow')
        label1.append("Country with male majority")
    else:
        womenmajority.append('red')
        label1.append("Country with female majority")
 

womenmajority1 = pd.DataFrame(womenmajority)
womenmajority1 = womenmajority1.rename(columns={0:'Plot Color % Female population'})

labelGender = pd.DataFrame(label1)
labelGender = labelGender.rename(columns={0:'Female majority label'})

mymerge_data = pd.merge(mymerge_data, womenmajority1, how="outer", left_index=True,right_index=True)
mymerge_data = pd.merge(mymerge_data, labelGender, how="outer", left_index=True,right_index=True)


#Compiling grabbed data from API Calls into one data frame

mymerge_data = pd.merge(mymerge, mymerge_data, how="outer", left_on=['country'],right_on=['Country'])
mymerge_data = mymerge_data.drop(['Country'],axis=1)
mymerge_data = mymerge_data.loc[mymerge_data['Total Unemployment'] > 0,
                                ['country','A2 (ISO)','Life Ladder','Total Unemployment','Individuals using the Internet (% of population)',
                                 '% Female Total Population','Mobile Phones (per 100 people)',
                                 'Plot Color % Female population','Female majority label']]
mymerge_data = mymerge_data.loc[mymerge_data['Life Ladder'] > 0,
                                ['country','A2 (ISO)','Life Ladder','Total Unemployment','Individuals using the Internet (% of population)',
                                 '% Female Total Population','Mobile Phones (per 100 people)',
                                 'Plot Color % Female population','Female majority label']]

mymerge_data

In [ ]:
#Saving all data to disk/file
outputfileCSV = open('Outputs/TotalUnemploymentVsHappinessPlot-Rafael.csv', 'w+' )
outputfile1 = mymerge_data.to_csv(index=True)
outputfileCSV.write(outputfile1)
outputfileCSV.close()


#Plotting 1
y_axis1 = mymerge_data['Life Ladder']       #use this for all plots
x_axis1 = mymerge_data['Total Unemployment']

Unemployment, ax = plt.subplots()
#Unemployment = plt.scatter(x_axis1,y_axis1,marker='o', alpha=0.75, color = mymerge_data['Plot Color % Female population'],edgecolor ="black", label = '% Total Unemployment')
Unemployment = plt.scatter(x_axis1,y_axis1,marker='o', alpha=0.75, color = 'orangered',edgecolor ="black", label = '% Total Unemployment')

# Set line for regression 
(slope, intercept,r_value, p_value, _) = scipy.stats.linregress(x_axis1, y_axis1)
fit = slope * x_axis1 + intercept


# Print R-Sqr and P-Value
print(f"r-squared: {r_value**2}, p-value: {p_value}")
# Plot line
ax.plot(x_axis1, fit, 'b--')


ax.legend(loc="best")
ax.grid(True)
plt.title("Unemployment x Happiness", fontsize=10, fontweight="bold")
plt.ylabel("Happiness index")
plt.xlabel("Unemployment")

#Save plot to file
Unemployment,plt.savefig("Graphs/TotalUnemploymentVsHappinessPlot-RafaelSantos.png")






#Plotting 2
y_axis1 = mymerge_data['Life Ladder']       #use this for all plots
x_axis2 = mymerge_data['% Female Total Population']

FemalePopulation, ax = plt.subplots()
#FemalePopulation = plt.scatter(x_axis2,y_axis1,marker='o', alpha=0.75, color = mymerge_data['Plot Color % Female population'],edgecolor ="black", label = '% Female Total Population')
FemalePopulation = plt.scatter(x_axis2,y_axis1,marker='o', alpha=0.75, color = 'lime',edgecolor ="black", label = '% Female Total Population')

# Set line for regression 
(slope, intercept,r_value, p_value, _) = scipy.stats.linregress(x_axis2, y_axis1)
fit2 = slope * x_axis2 + intercept


# Print R-Sqr and P-Value
print(f"r-squared: {r_value**2}, p-value: {p_value}")
# Plot line
ax.plot(x_axis2, fit2, 'b--')


ax.legend(loc="best")
ax.grid(True)
plt.title("% Female Total Population x Happiness", fontsize=10, fontweight="bold")
plt.ylabel("Happiness index")
plt.xlabel("% Female Total Population")

#Save plot to file
FemalePopulation,plt.savefig("Graphs/PercentageFemaleTotalPopulationVsHappinessPlot-RafaelSantos.png")




#Plotting 3

mymerge_dataInternet = mymerge_data.loc[mymerge_data['Individuals using the Internet (% of population)'] > 0,
                                ['country','A2 (ISO)','Life Ladder','Total Unemployment','Individuals using the Internet (% of population)',
                                 '% Female Total Population','Mobile Phones (per 100 people)',
                                 'Plot Color % Female population','Female majority label']]



y_axis1 = mymerge_dataInternet['Life Ladder']       #use this for all plots
x_axis3 = mymerge_dataInternet['Individuals using the Internet (% of population)']

Internet, ax = plt.subplots()
#Internet = plt.scatter(x_axis3,y_axis1,marker='o', alpha=0.75, color = mymerge_data['Plot Color % Female population'],edgecolor ="black", label = 'Individuals using the Internet (% of population)')
Internet = plt.scatter(x_axis3,y_axis1,marker='o', alpha=0.75, color = "gold",edgecolor ="black", label = 'Individuals using the Internet (% of population)')

# Set line for regression 
(slope, intercept,r_value, p_value, _) = scipy.stats.linregress(x_axis3, y_axis1)
fit3 = slope * x_axis3 + intercept


# Print R-Sqr and P-Value
print(f"r-squared: {r_value**2}, p-value: {p_value}")
# Plot line
ax.plot(x_axis3, fit3, 'b--')


ax.legend(loc="best")
ax.grid(True)
plt.title("Individuals using the Internet (% of population) x Happiness", fontsize=10, fontweight="bold")
plt.ylabel("Happiness index")
plt.xlabel("Individuals using the Internet (% of population)")

#Save plot to file
Internet,plt.savefig("Graphs/IndividualsUsingInternetVsHappinessPlot-RafaelSantos.png")




#Plotting 4
mymerge_dataMobile = mymerge_data.loc[mymerge_data['Mobile Phones (per 100 people)'] > 0,
                                ['country','A2 (ISO)','Life Ladder','Total Unemployment','Individuals using the Internet (% of population)',
                                 '% Female Total Population','Mobile Phones (per 100 people)',
                                 'Plot Color % Female population','Female majority label']]


y_axis1 = mymerge_dataMobile['Life Ladder']       #use this for all plots
x_axis4 = mymerge_dataMobile['Mobile Phones (per 100 people)']

MobilePhones100ppl, ax = plt.subplots()
#MobilePhones100ppl = plt.scatter(x_axis4,y_axis1,marker='o', alpha=0.75, color = mymerge_dataMobile['Plot Color % Female population'],edgecolor ="black", label = 'Mobile Phones (per 100 people)')
MobilePhones100ppl = plt.scatter(x_axis4,y_axis1,marker='o', alpha=0.75, color = 'tomato',edgecolor ="black", label = 'Mobile Phones (per 100 people)')

# Set line for regression 
(slope, intercept,r_value, p_value, _) = scipy.stats.linregress(x_axis4, y_axis1)
fit4 = slope * x_axis4 + intercept


# Print R-Sqr and P-Value
print(f"r-squared: {r_value**2}, p-value: {p_value}")
# Plot line
ax.plot(x_axis4, fit4, 'b--')


ax.legend(loc="best")
ax.grid(True)
plt.title("Mobile Phones (per 100 people) x Happiness", fontsize=10, fontweight="bold")
plt.ylabel("Happiness index")
plt.xlabel("Mobile Phones (per 100 people)")

#Save plot to file
MobilePhones100ppl,plt.savefig("Graphs/MobilePhones100pplVsHappinessPlot-RafaelSantos.png")

Arghavan

In [ ]:
#Cleaning Happiness DF
Happiness =  pd.read_csv('Resources/WorldHappinessReport.csv')
Happiness_New = Happiness.drop(columns=['Delivery Quality', 'Democratic Quality', 
                                        'Standard deviation of ladder by country-year',
                                        'Standard deviation/Mean of ladder by country-year',
                                        'GINI index (World Bank estimate)', 'GINI index (World Bank estimate), average 2000-15',
                                        'gini of household income reported in Gallup, by wp5-year'])

Happiness_Updated = Happiness_New.loc[Happiness_New["year"]==2016,:]
Happiness_Olympics = Happiness_Updated.dropna()

In [ ]:
#Cleaning Olympics DF
CSV_athletes = pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
CSV_countries =  pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")

#Cleaning Athletes DF
CSV_athletes = CSV_athletes.drop(columns=['Season', 'City', 'Height', 'Weight'])
CSV_athletes = CSV_athletes.loc[CSV_athletes["Year"]==2016,:]
CSV_athletes_updated = CSV_athletes.fillna("none")

#Merging Athletes and Country DF
merged_athletes = pd.merge(CSV_athletes_updated, CSV_countries, on = "NOC", how = "left")
merged_athletes = merged_athletes.rename(columns= {"region" : "country"})

#Getting Number of Atheletes
merged_athletes1 = merged_athletes.groupby(merged_athletes["country"])
Final_athletes = merged_athletes1["Name"].count()
Final_df = pd.DataFrame(Final_athletes)
Final_df = Final_df.rename(columns= {"Name" : "Number of Athletes"})

# Changin USA to United States so it merges with Happiness DF
Final_df = Final_df.rename(index={"USA":'United States'})

In [ ]:
#Creating Final DF
Athlete_Happiness = pd.merge(Final_df, Happiness_Olympics, on = "country")
Athlete_Happiness.to_csv("Outputs/Athlete_Happiness.csv")

Shayan

In [ ]:
happiness_df = pd.read_csv("Resources/WorldHappinessReport.csv")
tourism_df = pd.read_csv("Resources/Tourism.csv", skiprows=3)
happiness17_df = happiness_df.loc[(happiness_df["year"] == 2017)]
happiness_score = happiness17_df[["country", "Life Ladder"]]
happiness_score.columns = ["Country Name", "Happines Score"]
tourism_number = tourism_df[["Country Name", "2017"]]
tourism_number = tourism_number.fillna(0)
merge_table = pd.merge(happiness_score, tourism_number, on = "Country Name")
merge_table.to_csv("Outputs/Tourism_Happiness.csv")


Mariana

In [ ]:
#create a dataframe from a file using pandas
file = 'Resources/Capitals.csv'
# read the CSV in pandas
weather = pd.read_csv(file, encoding="ISO-8859-1")
weather.reset_index(inplace=True)
weather.columns

In [ ]:
# set up additional columns to hold information
weather['Temperature'] = ""

base_url = "http://api.openweathermap.org/data/2.5/weather?"

#print(response['main']['temp'])
# use iterrows to iterate through pandas dataframe
for index, row in weather.iterrows():

    # iterate thru each row of the file
    city = row['Cities']
    url = base_url + "appid=" + "082ff2cde1d396b2b896b2335daad199" + "&q=" + city +"&units=imperial"

# assemble url and make API request
    response = requests.get(url).json()
  
    #update the dataframe 
    try:
        weather.loc[index, 'Temperature'] = response['main']['temp']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
weather.head()

Sandra

1. get top songs by country choosing one random day a month for the whole year of 2017

In [ ]:
#create a dataframe from a file using pandas
file = 'Resources/CountryCodesLookup.csv'
#upload country lookup
Countries=pd.read_csv(file,encoding="ISO-8859-1")
#upload dates to look for top songs
spDates= pd.read_csv('Resources/DatesForSpotifyTopSongs.csv')
Countries.columns

In [ ]:
#filter by countries in spotify
spCountries = Countries.loc[Countries["spotifyFlag"] == 1,["COUNTRY","A2 (ISO)"]]

# make a list of countries by dates to get their top 200 songs
#cross join
spCountries["Key"]=1
spDates["Key"]=1
spCountries["Processed"]=0
CountryDate = pd.merge(spCountries, spDates, on="Key").reset_index(drop=True)
#.head(1)
#CountryDate

In [ ]:
#extract top songs by country in 2017. 
# rows get marked as processed. If the script failes, at re-start, it will exclude processed/failed rows.
# this needs to be restarted as many times as it fails.

spCountriesToProcess = CountryDate.loc[(CountryDate["Processed"] == 0) & (CountryDate["COUNTRY"]!='AD' ),:]


from fycharts import SpotifyCharts 
api = SpotifyCharts.SpotifyCharts()


# use iterrows to iterate through pandas dataframe
for index, row in spCountriesToProcess.iterrows():

    # iterate thru each row of the file
    countryCode = str.lower(row['A2 (ISO)'])
    startDate=row["startDate"]
    EndDate=row["EndDate"]
    
    #get data
    print(startDate)  
    print (countryCode)
    CountryDate.loc[index, 'Processed'] = 1
   
       
    api.top200Daily(output_file = 'Outputs/Spotify2017Top.csv', start=startDate, end=EndDate, region=countryCode)
#viral50Daily    
#top200Weekly


2. Query spotify API. Find the songs attributes.  Classify all songs found and saved in the Soptify2017Top.cvs 

In [ ]:
file='Outputs/Spotify2017Top1.csv'
TopSpotify=pd.read_csv(file,encoding="ISO-8859-1")
TopSpotify.columns
TopSpotify["uri"]='spotify:track:'+TopSpotify["id"]
#New columns
TopSpotify["happiness"]=''
TopSpotify["energy"]=''
TopSpotify["danceability"]=''
TopSpotify["loudness"]=''
TopSpotify["speechiness"]=''
TopSpotify["instrumentalness"]=''


TopSpotify.head(1)

In [ ]:

import spotipy

client_id='hiddenCode'
client_secret='HiddenCode'

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



# use iterrows to iterate through pandas dataframe
for index, row in TopSpotify.iterrows():

    # iterate thru each row of the file
    tid = row['uri']
    try:
        fearure = sp.audio_features(tid)
        happiness = fearure[0]["valence"]
        energy = fearure[0]["energy"]
        danceability=fearure[0]["danceability"]
        loudness=fearure[0]["loudness"]
        speechiness=fearure[0]["speechiness"]
        instrumentalness=fearure[0]["instrumentalness"]

    #write back to data frame
               
        TopSpotify.loc[index, 'happiness'] = happiness
        TopSpotify.loc[index, 'energy'] = energy
        TopSpotify.loc[index, 'danceability'] = danceability
        TopSpotify.loc[index, 'loudness'] = loudness
        TopSpotify.loc[index, 'speechiness'] = speechiness
        TopSpotify.loc[index, 'instrumentalness'] = instrumentalness
        

    except (KeyError, IndexError, NoneType):
        print(tid)
        print("Missing field/result... skipping.")
TopSpotify.to_csv("Outputs/SpotifyTop2017Final.csv", index=False)